# Delta Water Unavailability Methodology Module 4: Curtailment Summary

In [ ]:
# This script calls the outputs of the Analysis script to summarize appropriate curtailments in the Sacramento-San Joaquin
# Delta Waterhsed based on the results of the water unavailability analysis
# Absent any active curtailment regulation, results are for informational purposes only and have no regulatory effect
# Further documentation is available on the Delta Water Unavailability Methodology website:
# https://www.waterboards.ca.gov/waterrights/water_issues/programs/drought/drought_tools_methods/delta_method.html

# Import necessary packages
import pandas as pd
import numpy as np
import os
from IPython.display import display, Markdown

In [ ]:
# Function to check if required input files are present
def check_required_outputs(folder_path, files_to_notebooks):
    if not os.path.exists(folder_path):
        print(f"Required folder 'intermediate-outputs' does not exist. Please run the Supply, Demand, and Analysis notebooks first.")
        return False

    missing_instructions = []
    for file, notebook in files_to_notebooks.items():
        if not os.path.exists(os.path.join(folder_path, file)):
            missing_instructions.append(f"{file} (Please run {notebook})")

    if missing_instructions:
        missing_str = "\n".join(missing_instructions)
        print(f"The following required files are missing:\n{missing_str}")
        return False

    print("All required intermediate outputs are present. Proceeding with analysis.")
    return True

In [ ]:
# Check if the intermediate outputs generated by previous scripts are present
files_to_notebooks = {
    'Demands_Period_POD.csv': 'Demand.ipynb',
    'Supply_Period_Total.csv': 'Supply.ipynb',
    'Headwater_Calculations.csv': 'Analysis.ipynb',
    'Analysis_Results.csv': 'Analysis.ipynb',
}

if not check_required_outputs('./intermediate-outputs', files_to_notebooks):
    display(Markdown("**Execution stopped. Please generate the required intermediate outputs before proceeding.**"))

In [ ]:
# Import datsets defining water rights, where they divert, and their demands
water_rights = pd.read_csv('../user-inputs/WaterRights.csv')
pods = pd.read_csv('../user-inputs/PODs.csv')
demands_dataset = pd.read_csv('./intermediate-outputs/Demands_Period_POD.csv')

# Import the results of the headwater and watershed-scale water unavailability analyses
supply_period_total = pd.read_csv('./intermediate-outputs/Supply_Period_Total.csv')
hwa_outputs = pd.read_csv('./intermediate-outputs/Headwater_Calculations.csv', index_col=0)
analysis_dataset = pd.read_csv('./intermediate-outputs/Analysis_Results.csv', header=1, usecols = ['APPL_ID','WATERSHED','SUBWATERSHED', 'PRIORITY_DATE_CUSTOM', 'LEGAL_DELTA', 'HEADWATER', 'WATER_UNAVAILABLE_HEADWATER', 'WATER_UNAVAILABLE_WATERSHED', 'DEMAND_MET_WATERSHED_TOTAL', 'DEMAND_UNMET_WATERSHED_TOTAL', 'DEMAND_UNMET_HEADWATER_TOTAL'], dtype={'PRIORITY_DATE_CUSTOM':'string', 'WATER_UNAVAILABLE_HEADWATER':'bool', 'WATER_UNAVAILABLE_WATERSHED':'bool', 'WATER_UNAVAILABLE_EITHER':'bool'})
# Import the same file again but keep the hierarchical columns
analysis_dataset_heir = pd.read_csv('./intermediate-outputs/Analysis_Results.csv', header = [0,1])

In [ ]:
# Grab results for the final record in the Analysis Results
# These represent conditions after water unavailability is analyzed for the most junior right in the watershed
supply_headwater_acct = analysis_dataset_heir.xs('SUPPLY_HEADWATER_ACCT', axis = 1, level = 0).iloc[-1]
demand_headwater = analysis_dataset_heir.xs('DEMAND_HEADWATER', axis = 1, level = 0).iloc[-1]
supply_watershed_acct = analysis_dataset_heir.xs('SUPPLY_WATERSHED_ACCT', axis = 1, level = 0).iloc[-1]
demand_watershed = analysis_dataset_heir.xs('DEMAND_WATERSHED', axis = 1, level = 0).iloc[-1]

In [ ]:
# Reorganize imported Headwater Calculations for easier processing
hwa_outputs = hwa_outputs.transpose()
hwa_outputs = hwa_outputs.rename_axis('SUBWATERSHED').reset_index()
hwa_outputs['SUPPLY_PERIOD_FINAL']=hwa_outputs['SUPPLY_PERIOD_FINAL'].astype(float)
hwa_outputs['SUPPLY_PERIOD_TOTAL']=hwa_outputs['SUPPLY_PERIOD_TOTAL'].astype(float)

In [ ]:
# Define common lists of subwatersheds
subwatersheds = ['Sacramento Bend', 'Stony', 'Cache', 'Upper Feather', 'Yuba', 'Bear', 'Upper American', 'Putah', 
                 'Upper Sacramento Valley', 'Sacramento Valley Floor', 'Chowchilla', 'Upper San Joaquin', 'Fresno',
                 'Merced', 'Tuolumne', 'Stanislaus', 'Calaveras', 'Mokelumne', 'Cosumnes', 'San Joaquin Valley Floor']
headwater_subwatershed = ['Sacramento Bend', 'Stony', 'Cache', 'Upper Feather', 'Yuba', 'Bear', 'Upper American', 'Putah', 
                 'Chowchilla', 'Upper San Joaquin', 'Fresno', 'Merced', 'Tuolumne', 'Stanislaus', 'Calaveras', 'Mokelumne',
                'Cosumnes']
summary_subwatersheds = ['Sacramento Bend', 'Stony', 'Cache', 'Upper Feather', 'Yuba', 'Bear', 'Upper American', 'Putah', 
                 'Upper Sacramento Valley', 'Sacramento Valley Floor', 'Chowchilla', 'Upper San Joaquin', 'Fresno',
                 'Merced', 'Tuolumne', 'Stanislaus', 'Calaveras', 'Mokelumne', 'Cosumnes', 'San Joaquin Valley Floor',
                'TOTAL']
sacramento_subwatersheds = ['Sacramento Bend', 'Upper Sacramento Valley', 'Stony', 'Cache', 'Upper Feather',
                            'Yuba', 'Bear', 'Upper American', 'Sacramento Valley Floor', 'Putah']
sanjoaquin_subwatersheds = ['Chowchilla', 'Upper San Joaquin', 'Fresno', 'Merced', 'Tuolumne', 
                            'Stanislaus', 'Calaveras', 'Mokelumne', 'Cosumnes', 'San Joaquin Valley Floor']

In [ ]:
'''
Create the Curtailment dataset by taking the Water Rights dataset (each row representing one water right or claim)
and assign attributes explaining where each diverts
'''

# From the Water Rights dataset, take the list of rights and each's Primary Owner, Water Right Type, and Priority Date
selected_columns = ['APPL_ID', 'PRIMARY_OWNER_NAME', 'WATER_RIGHT_TYPE_CUSTOM', 'PRIORITY_DATE_CUSTOM']
water_rights = water_rights.dropna()
curtailment_dataset = water_rights[selected_columns].copy()

# Creating mappings for the PODs dataset
pods_first_occurrence = pods.drop_duplicates(subset='APPL_ID')
pods_mapping = pods_first_occurrence.set_index('APPL_ID')[['WATERSHED', 'SUBWATERSHED', 'LEGAL_DELTA']].to_dict('index')

# Creating mappings for the Analysis dataset
analysis_mapping = analysis_dataset.groupby('APPL_ID').agg({
    'WATERSHED': lambda x: 'Both' if x.nunique() > 1 else x.iloc[0],# Indicate if demands exist in Both Sacramento and San Joaquin watersheds
    'SUBWATERSHED': lambda x: 'Multiple' if x.nunique() > 1 else x.iloc[0],# Indicate if demands exist in Multiple subwatersheds
    'LEGAL_DELTA': lambda x: 'Partial' if x.nunique() > 1 else x.iloc[0],# Indicate if demands exist Partial(ly) in the Legal Delta
    'HEADWATER': lambda x: 'Partial' if x.nunique() > 1 else x.iloc[0]# Indicate if demands exist Partial(ly) in a Headwater subwatershed
}).to_dict('index')

def process_curtailment_row(row):
    appl_id = row['APPL_ID']
    priority_date = row['PRIORITY_DATE_CUSTOM']
    # Information on Riparian-priority claims is only found in the PODs dataset
    if priority_date == 'Riparian':
        pods_row = pods_mapping.get(appl_id, {'WATERSHED': 'Unknown', 'SUBWATERSHED': 'Unknown', 'LEGAL_DELTA': 'Unknown'})
        watershed, subwatershed, legal_delta = pods_row['WATERSHED'], pods_row['SUBWATERSHED'], pods_row['LEGAL_DELTA']
        headwater = False if subwatershed in ['Upper Sacramento Valley', 'Sacramento Valley Floor', 'San Joaquin Valley Floor'] else True
    # Information on all other claims and rights was already assembled in the Analysis dataset
    else:
        analysis_row = analysis_mapping.get(appl_id, {'WATERSHED': 'Unknown', 'SUBWATERSHED': 'Unknown', 'LEGAL_DELTA': 'Unknown', 'HEADWATER': 'Unknown'})
        watershed, subwatershed, legal_delta, headwater = analysis_row['WATERSHED'], analysis_row['SUBWATERSHED'], analysis_row['LEGAL_DELTA'], analysis_row['HEADWATER']

    return watershed, subwatershed, legal_delta, headwater

curtailment_dataset[['WATERSHED', 'SUBWATERSHED', 'LEGAL_DELTA', 'HEADWATER']] = curtailment_dataset.apply(
    lambda row: process_curtailment_row(row), axis=1, result_type='expand'
)

In [ ]:
'''
For each right, determine Unavailability at the Headwater scale
'''
# Map each SUBWATERSHED to its corresponding SUPPLY_PERIOD_TOTAL to avoid repeated DataFrame lookups
supply_period_totals = hwa_outputs.groupby('SUBWATERSHED')['SUPPLY_PERIOD_TOTAL'].first().to_dict()
# Map each APPL_ID to a boolean indicating overall headwater unavailability status
water_unavailable_headwater = analysis_dataset.groupby('APPL_ID')['WATER_UNAVAILABLE_HEADWATER'].all().to_dict()

def determine_unavailability_headwater(row):
    # Unavailability to Riparian claims is unique
    if row['PRIORITY_DATE_CUSTOM'] == 'Riparian' and not row['LEGAL_DELTA'] and row['HEADWATER']:
        # Check unavailability for Riparian rights in the specified SUBWATERSHED
        supply_period_final_value = supply_period_totals.get(row['SUBWATERSHED'], 0)
        return supply_period_final_value == 0
    elif water_unavailable_headwater.get(row['APPL_ID'], False):
        return True
    return False

curtailment_dataset['UNAVAILABILITY_HEADWATER'] = curtailment_dataset.apply(
    determine_unavailability_headwater, axis=1)

In [ ]:
'''
For each right, determine Unavailability at the Watershed scale
'''

# Aggregated 'SUPPLY_PERIOD_FINAL' totals for defined subwatershed groups.
aggregated_supply_period_final = {
    'sacramento': hwa_outputs.loc[hwa_outputs['SUBWATERSHED'].isin(sacramento_subwatersheds), 'SUPPLY_PERIOD_FINAL'].sum(),
    'sanjoaquin': hwa_outputs.loc[hwa_outputs['SUBWATERSHED'].isin(sanjoaquin_subwatersheds), 'SUPPLY_PERIOD_FINAL'].sum(),
    'total': hwa_outputs['SUPPLY_PERIOD_FINAL'].sum(),
}
# Individual 'SUPPLY_PERIOD_FINAL' values for each subwatershed outside headwater
individual_supply_period_final = {
    'Upper Sacramento Valley': hwa_outputs.loc[hwa_outputs['SUBWATERSHED'].isin(['Sacramento Bend', 'Upper Sacramento Valley']), 'SUPPLY_PERIOD_FINAL'].sum(),
    'Sacramento Valley Floor': hwa_outputs.loc[hwa_outputs['SUBWATERSHED'].isin(['Sacramento Bend', 'Upper Sacramento Valley', 'Stony', 'Cache', 'Upper Feather', 
                                                                                 'Yuba', 'Bear', 'Upper American', 'Sacramento Valley Floor']), 'SUPPLY_PERIOD_FINAL'].sum(),
    'San Joaquin Valley Floor': hwa_outputs.loc[hwa_outputs['SUBWATERSHED'].isin(['Upper San Joaquin', 'Fresno', 'Chowchilla', 'Merced', 
                                                                                  'Tuolomne', 'San Joaquin Valley Floor']), 'SUPPLY_PERIOD_FINAL'].sum(),
}

# Maps each APPL_ID to a boolean indicating if water is unavailable at the headwater.
water_unavailable_headwater = analysis_dataset.groupby('APPL_ID')['WATER_UNAVAILABLE_HEADWATER'].all().to_dict()

# Maps each APPL_ID to a boolean indicating if water is unavailable at the watershed for non-Riparian rights.
non_riparian_unavailability = analysis_dataset[analysis_dataset['PRIORITY_DATE_CUSTOM'] != 'Riparian'
                                              ].groupby('APPL_ID')['WATER_UNAVAILABLE_WATERSHED'].all().to_dict()

# Function to determine watershed-scale unavailability
def determine_unavailability_watershed(row):
    if water_unavailable_headwater.get(row['APPL_ID'], False):
        return False
    # Unavailability to Riparian claims is unique
    if row['PRIORITY_DATE_CUSTOM'] == 'Riparian':
        # Legal Delta riparian unavailability is based on total supplies from upstream subwatersheds
        if row['LEGAL_DELTA']:
            # Check unavailability for Riparian rights within the Legal Delta
            if row['WATER_RIGHT_TYPE_CUSTOM'] == 'Statement of Div and Use (Riparian)':
                if row['WATERSHED'] == 'Sacramento':
                    return aggregated_supply_period_final['sacramento'] == 0
                elif row['WATERSHED'] == 'San Joaquin':
                    return aggregated_supply_period_final['sanjoaquin'] == 0
            else:
                return aggregated_supply_period_final['total'] == 0
        elif not row['LEGAL_DELTA'] and row['HEADWATER']:
            subwatershed_supply = hwa_outputs.loc[
                hwa_outputs['SUBWATERSHED'] == row['SUBWATERSHED'], 'SUPPLY_PERIOD_TOTAL'].sum()
            return subwatershed_supply == 0
        elif not row['LEGAL_DELTA'] and not row['HEADWATER']:
            return individual_supply_period_final.get(row['SUBWATERSHED'], 0) == 0
    else:
        return non_riparian_unavailability.get(row['APPL_ID'], False)

    return False

# Apply the function
curtailment_dataset['UNAVAILABILITY_WATERSHED'] = curtailment_dataset.apply(determine_unavailability_watershed, axis=1)

In [ ]:
'''
For each right, determine if Water Unavailable at Either the Headwater or Watershed scale 
'''

curtailment_dataset['UNAVAILABILITY_EITHER'] = curtailment_dataset.apply(
    lambda row: row['UNAVAILABILITY_HEADWATER'] or row['UNAVAILABILITY_WATERSHED'], axis=1)

In [ ]:
'''
For each right, determine a Curtailment Status based on water unavailability
'''
''' Pending Water Right Types are unlawful claims that are never authorized to divert
    Curtailment Status is based on water unavailabity at either the headwater or watershed scale
    Periodically during the emergency regulation, the Deputy Director for Water Rights exercised discretion
    to base curtailments only on the watershed-scale analysis (UNAVAILABILITY_WATERSHED)
'''

curtailment_dataset['CURTAILMENT_STATUS'] = curtailment_dataset.apply(
    lambda row: 'Not Authorized to Divert' if row['PRIORITY_DATE_CUSTOM'] == 'Pending' else ('Curtailed' if row['UNAVAILABILITY_EITHER'] else 'Not Curtailed'), axis=1)

## TO DO: build additional functionality based on other aspects of water rights
## e.g., under the emergency regulation Cannabis Registrations were Not Authorized to Divert during the forebearance season

In [ ]:
'''
For each right, calculate a total Demand (Direct + Storage for the user-specified period)
'''
demands_sum = demands_dataset.groupby('APPL_ID').agg({
    'DIRECT_PERIOD_POD': 'sum',
    'STORAGE_PERIOD_POD': 'sum'
}).reset_index()
demands_sum['TOTAL_DEMAND'] = demands_sum['DIRECT_PERIOD_POD'] + demands_sum['STORAGE_PERIOD_POD']

curtailment_dataset = curtailment_dataset.merge(demands_sum[['APPL_ID', 'TOTAL_DEMAND']].rename(columns={'TOTAL_DEMAND': 'DEMAND_CURTAILMENT'}), on='APPL_ID', how='left')
curtailment_dataset['DEMAND_CURTAILMENT'].fillna(0, inplace=True)

In [ ]:
'''
For each right, calculate a total Demand Met
'''
demand_met_total = analysis_dataset.groupby('APPL_ID')['DEMAND_MET_WATERSHED_TOTAL'].sum().reset_index()

# Merges curtailment records with aggregated demand met totals by APPL_ID for precise DEMAND_MET_CURTAILMENT calculation.
temp_df = curtailment_dataset.merge(demand_met_total, on='APPL_ID', how='left')

curtailment_dataset['DEMAND_MET_CURTAILMENT'] = np.where(
    curtailment_dataset['PRIORITY_DATE_CUSTOM'] == "Riparian",
    np.where(curtailment_dataset['UNAVAILABILITY_EITHER'], 0, curtailment_dataset['DEMAND_CURTAILMENT']),
    temp_df['DEMAND_MET_WATERSHED_TOTAL'].fillna(0)
)

In [ ]:
'''
For each right, determine if there is Partial Unavailability
(i.e., some water is available but not 100% of demand can be met)
'''

curtailment_dataset['UNAVAILABILITY_PARTIAL'] = curtailment_dataset.apply(
    lambda row: row['DEMAND_CURTAILMENT'] > 0 and row['DEMAND_MET_CURTAILMENT'] < row['DEMAND_CURTAILMENT'],
    axis=1
)

In [ ]:
# Export the Curtailment Data
curtailment_dataset.to_csv('./output-data/Curtailment_Data.csv', index=False)

In [ ]:
'''
Develop a Curtailment Details Summary table for each of the 20 Subwatersheds, the Legal Delta, and a Total value
'''

# Function to calculate various summary values
def calculate_summary_values(subwatershed):
    headwater_priority_date = ""
    watershed_priority_date = ""
    demand_unmet_headwater = 0
    demand_unmet_watershed = 0
    excess_supply_headwater = 0
    excess_supply_watershed = 0
    num_curtailments = 0
    demand_curtailment = 0
    
    # Filter Curtailment dataset for the current subwatershed
    subwatershed_curtailment = curtailment_dataset.loc[curtailment_dataset['SUBWATERSHED'] == subwatershed]
    
    # Determine Headwater Priority Date of First Curtailment
    # (the most senior right or claim where Water Unavailable in Headwater is true)
    if subwatershed in headwater_subwatershed:
        sacramento_total_supply = hwa_outputs[hwa_outputs['SUBWATERSHED'].isin(sacramento_subwatersheds)]['SUPPLY_PERIOD_TOTAL'].sum()
        if sacramento_total_supply == 0:
            headwater_priority_date = 'Riparian'
        else:
            # Check if there is any unavailability at the headwater scale
            headwater_filtered_df = analysis_dataset.loc[(analysis_dataset['SUBWATERSHED'] == subwatershed) & 
                                                           (analysis_dataset['WATER_UNAVAILABLE_HEADWATER'] == True)]
            if not headwater_filtered_df.empty:
                headwater_priority_date = headwater_filtered_df['PRIORITY_DATE_CUSTOM'].iloc[0]
            else:
                # If there are no curtailments at the headwater scale, show "-"
                headwater_priority_date = '-'
    
    # For headwater subwatersheds, show "N/A"
    elif subwatershed not in headwater_subwatershed:
        headwater_priority_date = 'N/A'

    # Determine Watershed Priority Date of First Curtailment
    if subwatershed == 'TOTAL':
        watershed_priority_date = 'N/A'
    else:
        watershed_filtered_df = analysis_dataset.loc[(analysis_dataset['SUBWATERSHED'] == subwatershed) & 
                                                     (analysis_dataset['WATER_UNAVAILABLE_WATERSHED'] == True)]
        if not watershed_filtered_df.empty:
            watershed_priority_date = watershed_filtered_df['PRIORITY_DATE_CUSTOM'].iloc[0]
        else:
            watershed_priority_date = '-'

    # Calculate Demand Unmet in Headwater Analysis
    if subwatershed in subwatersheds:
        demand_unmet_headwater = analysis_dataset.loc[(analysis_dataset['SUBWATERSHED'] == subwatershed) & 
                                                      (analysis_dataset['LEGAL_DELTA'] == False)]['DEMAND_UNMET_HEADWATER_TOTAL'].sum()
    elif subwatershed == 'Total':
        demand_unmet_headwater = analysis_dataset['DEMAND_UNMET_HEADWATER_TOTAL'].sum()
    
    # Calculate Demand Unmet in Watershed Analysis
    if subwatershed in subwatersheds:
        demand_unmet_watershed = analysis_dataset.loc[(analysis_dataset['SUBWATERSHED'] == subwatershed) & 
                                                      (analysis_dataset['LEGAL_DELTA'] == False)]['DEMAND_UNMET_WATERSHED_TOTAL'].sum()
    elif subwatershed == 'Total':
        demand_unmet_watershed = analysis_dataset['DEMAND_UNMET_WATERSHED_TOTAL'].sum()

    # Calculate Excess Supply in Headwater Analysis
    if subwatershed in headwater_subwatershed:
        excess_supply_headwater = max(0, (supply_headwater_acct[subwatershed] - demand_headwater[subwatershed]))
    elif subwatershed == 'Total':
        excess_supply_headwater = max(0, (supply_headwater_acct.sum() - demand_headwater.sum()))
    else:
        excess_supply_headwater = 'N/A'
    
    # Calculate Excess Supply in Watershed Analysis
    if subwatershed in subwatersheds:
        excess_supply_watershed = max(0, (supply_watershed_acct[subwatershed] - demand_watershed[subwatershed]))
    elif subwatershed == 'Total':
        excess_supply_watershed = max(0, (supply_watershed_acct.sum() - demand_watershed.sum()))

    # Calculate Number of Curtailments
    if subwatershed in subwatersheds:
        num_curtailments = len(subwatershed_curtailment[(subwatershed_curtailment['CURTAILMENT_STATUS'] == "Curtailed") & 
                                                         (subwatershed_curtailment['LEGAL_DELTA'] == False)])
    elif subwatershed == 'Total':
        num_curtailments = len(subwatershed_curtailment[subwatershed_curtailment['CURTAILMENT_STATUS'] == "Curtailed"])
    
    # Calculate Demand of Curtailed Rights
    if subwatershed in subwatersheds:
        demand_curtailment = subwatershed_curtailment[(subwatershed_curtailment['CURTAILMENT_STATUS'] == "Curtailed") & 
                                                      (subwatershed_curtailment['LEGAL_DELTA'] == False)]['DEMAND_CURTAILMENT'].sum()
    elif subwatershed == 'Total':
        demand_curtailment = subwatershed_curtailment[(subwatershed_curtailment['CURTAILMENT_STATUS'] == "Curtailed")]['DEMAND_CURTAILMENT'].sum()
    
    return [subwatershed, headwater_priority_date, watershed_priority_date, demand_unmet_headwater, 
            demand_unmet_watershed, excess_supply_headwater, excess_supply_watershed, 
            num_curtailments, demand_curtailment]

summary_list = []    

# Execute the function for all subwatersheds
for subwatershed in summary_subwatersheds:
    summary_dict = calculate_summary_values(subwatershed)
    summary_list.append(summary_dict)

# Convert the list of summary values to a dataframe
curtailment_summary = pd.DataFrame(summary_list, columns=['Subwatershed', 
                                           'Headwater Priority Date of First Curtailment',
                                           'Watershed Priority Date of First Curtailment',
                                           'Demand Unmet in Headwater Analysis', 
                                           'Demand Unmet in Watershed Analysis',
                                           'Excess Supply in Headwater Analysis',
                                           'Excess Supply in Watershed Analysis',
                                           'Number of Curtailments',
                                           'Demand of Curtailed Rights'])

# Convert specific columns to integer for display
int_columns = ['Demand Unmet in Headwater Analysis', 
               'Demand Unmet in Watershed Analysis',
               'Excess Supply in Headwater Analysis',
               'Excess Supply in Watershed Analysis',
               'Number of Curtailments',
               'Demand of Curtailed Rights']

for col in int_columns:
    # Check if the column is not of a string type to avoid errors
    if curtailment_summary[col].dtype != 'object':
        curtailment_summary[col] = np.ceil(curtailment_summary[col].fillna(0)).astype(int)

In [ ]:
# Display and export the Curtailment Details Summary table
display(curtailment_summary)
curtailment_summary.to_csv('./output-data/Curtailment_Summary.csv', index=False)

In [ ]:
'''
Develop a Project Coordinated Operations Agreement (COA) Rights Summary table
In recognition of the COA’s provisions for sharing limited Project supplies,
these rights were only curtailed if water was unavailable to all of them in the watershed-scale analysis
(consistent with curtailment implementation since the 6/27/2022 Methodology update)
'''

# Filter the DataFrame to include only rows where 'PRIORITY_DATE_CUSTOM' is 'Project'
project_coa_rights_df = curtailment_dataset[curtailment_dataset['PRIORITY_DATE_CUSTOM'] == 'Project']

# Extract the list of Application IDs for 'Project' rights
project_coa_rights = project_coa_rights_df['APPL_ID'].unique().tolist()

# Look up Unavailability (at Either Headwater or Watershed scale) for each right
project_coa_curtailment = curtailment_dataset[curtailment_dataset['APPL_ID'].isin(project_coa_rights)].copy()
water_unavailable_to_all = project_coa_curtailment['UNAVAILABILITY_EITHER'].all()
    
# Set the CURTAILMENT_STATUS for all Project COA rights
project_coa_curtailment.loc[:, 'CURTAILMENT_STATUS'] = np.where(
    project_coa_curtailment['UNAVAILABILITY_EITHER'], "Curtailed", "Not Curtailed"
)

# Create the Project COA Rights Summary Table
project_coa_summary = project_coa_curtailment[['APPL_ID', 'UNAVAILABILITY_EITHER', 'CURTAILMENT_STATUS']]

# Display the Project COA Rights Summary Table and Water Unavailable to All status
print("Project COA Rights Summary:")
display(project_coa_summary)
print("\nWater Unavailable to All Project COA Rights:", water_unavailable_to_all)

In [ ]:
# Export the Project COA Rights Summary table
project_coa_summary.to_csv('./output-data/Project_Rights_Summary.csv', index=False)